In [2]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np

In [3]:
IMAGE_PATH = 'Tensorflow\\workspace\\images\\test\\110.png'
CHECKPOINT_PATH = 'Tensorflow\\workspace\\models\\bingus_model\\ckpt-4'
PIPELINE_CONFIG_PATH = 'Tensorflow\\workspace\\models\\bingus_model\\pipeline.config'

In [ ]:
def load_model(pipeline_config_path, ckpt_path):
    # Load pipeline config and build a detection model
    configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
    detection_model = model_builder.build(model_config=configs['model'], is_training=False)

    # Restore checkpoint
    ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
    ckpt.restore(ckpt_path).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

def label_image(img_path):
    img = cv2.imread(img_path)
    image_np = np.array(img)
    y_h = image_np.shape[0]
    x_h = image_np.shape[1]

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    # print(detections)
    # print(len(detections))

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    final_detections = []
    for i in range(detections['num_detections']):
        if detections['detection_scores'][i] > THRESHOLD:
            dbox = np.copy(detections['detection_boxes'][i])
            dbox[0] *= y_h
            dbox[1] *= x_h
            dbox[2] *= y_h
            dbox[3] *= x_h
            final_detections.append({
                'label': detections['detection_classes'][i] + label_id_offset,
                'box_norm': detections['detection_boxes'][i],
                'box': dbox.astype(int),
                'score': detections['detection_scores'][i]
                })
    return final_detections


In [4]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(CHECKPOINT_PATH).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [5]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)
y_h = image_np.shape[0]
x_h = image_np.shape[1]

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
# print(detections)
# print(len(detections))

label_id_offset = 1
image_np_with_detections = image_np.copy()

In [6]:
final_detections = []
THRESHOLD = 0.3
for i in range(detections['num_detections']):
    if detections['detection_scores'][i] > THRESHOLD:
        dbox = np.copy(detections['detection_boxes'][i])
        dbox[0] *= y_h
        dbox[1] *= x_h
        dbox[2] *= y_h
        dbox[3] *= x_h
        final_detections.append({
            'label': detections['detection_classes'][i] + label_id_offset,
            'box_norm': detections['detection_boxes'][i],
            'box': dbox.astype(int),
            'score': detections['detection_scores'][i]
            })

In [7]:
final_detections

[{'label': 3,
  'box_norm': array([0.43824703, 0.41720623, 0.5775799 , 0.5425698 ], dtype=float32),
  'box': array([ 773, 1593, 1018, 2072]),
  'score': 0.99421114},
 {'label': 1,
  'box_norm': array([0.        , 0.3701775 , 0.36916012, 0.6272964 ], dtype=float32),
  'box': array([   0, 1414,  651, 2396]),
  'score': 0.9936568},
 {'label': 3,
  'box_norm': array([0.03292408, 0.21814719, 0.16734695, 0.34571692], dtype=float32),
  'box': array([  58,  833,  295, 1320]),
  'score': 0.99266666},
 {'label': 3,
  'box_norm': array([0.8016169 , 0.39198196, 0.9586301 , 0.5374013 ], dtype=float32),
  'box': array([1414, 1497, 1691, 2052]),
  'score': 0.9909662},
 {'label': 1,
  'box_norm': array([0.27778032, 0.002736  , 0.790285  , 0.2558999 ], dtype=float32),
  'box': array([ 490,   10, 1394,  977]),
  'score': 0.989134},
 {'label': 3,
  'box_norm': array([0.03168967, 0.65816694, 0.17212266, 0.7855497 ], dtype=float32),
  'box': array([  55, 2514,  303, 3000]),
  'score': 0.98225397},
 {'label